## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.
2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).
3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.
Następnie znaleźć minimum.
4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.
6. Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np

if not os.path.exists("jet.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/jet.bmp --no-check-certificate
jet = cv2.imread("jet.bmp", cv2.IMREAD_GRAYSCALE)

In [ ]:
def BBHE_equalization(image):
    I = image
    H = cv2.calcHist([I],[0],None,[256],[0,256])
    lm = int(np.round(np.mean(I)))
    H1 = H[:lm]
    H2 = H[lm:]
    C1 = H1.cumsum()
    C1 = C1/np.max(C1)
    C2 = H2.cumsum()
    C2 = C2/np.max(C2)
    C1n = lm * C1
    C2n = lm + 1 +(255 - lm - 1) * C2
    C = np.concatenate([C1n, C2n])
    I_eq = cv2.LUT(I,C)
    return I_eq

In [ ]:
def DSIHE_equalization(image):
    I = image
    H = cv2.calcHist([I],[0],None,[256],[0,256])
    lm = int(np.round(np.median(I)))
    H1 = H[:lm]
    H2 = H[lm:]
    C1 = H1.cumsum()
    C1 = C1/np.max(C1)
    C2 = H2.cumsum()
    C2 = C2/np.max(C2)
    C1n = lm * C1
    C2n = lm + 1 +(255 - lm - 1) * C2
    C = np.concatenate([C1n, C2n])
    I_eq = cv2.LUT(I,C)
    return I_eq

In [ ]:
def show_image_hist_histcum(I, title):
    fig, ax = plt.subplots(1,3, figsize=(15,5))
    fig.suptitle(title,fontsize=20)
    ax[0].imshow(I,'gray',vmin=0,vmax=256)
    ax[0].axis('off')
    H = cv2.calcHist([I],[0],None,[256],[0,256])
    ax[1].plot(H)
    ax[1].set_title('Histogram')
    ax[1].grid('on')
    H_cum = H.cumsum()
    ax[2].plot(H_cum)
    ax[2].set_title('Histogram skumulowany')
    ax[2].grid('on')
    plt.show()

In [ ]:
show_image_hist_histcum(jet,'Ogyginalny obraz')
show_image_hist_histcum(cv2.equalizeHist(jet),'Klasyczne wyrównanie')
show_image_hist_histcum(BBHE_equalization(jet),'Wyrównanie BBHE')
show_image_hist_histcum(DSIHE_equalization(jet),'Wyrównanie DSIHE')